In [ ]:
from IPython.display import HTML




In [ ]:
# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
    position: relative;
}

.header {
    width: 100%;
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.header img {
    max-height: 100px;
}

.header h1 {
    flex-grow: 1;
    text-align: center;
    font-size: 6em;
    margin: 0;
}

.scroll-button {
    position: absolute;
    bottom: 40px;
    left: 50%;
    transform: translateX(-50%);
    background-color: #4CAF50;
    color: white;
    border: none;
    border-radius: 50%;
    width: 50px;
    height: 50px;
    display: flex;
    justify-content: center;
    align-items: center;
    font-size: 2em;
    cursor: pointer;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

# Header HTML with button
header_html = '''
<div class="centered-content">
    <div class="header">
        <img src="logo.png" alt="COEX Logo">
        <h1>Correlation and Regression</h1>
    </div>
    <button class="scroll-button" onclick="scrollToSection()">↓</button>
</div>
<script>
function scrollToSection() {
    var section = document.getElementById('target-section');
    if (section) {
        section.scrollIntoView({ behavior: 'smooth' });
    }
}
</script>
'''
display(HTML(header_html))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import os

# File path for the CSV
csv_file_path = "cix.csv"

# Create the necessary widgets
string_box = widgets.Text(description='Ticker')
description_box = widgets.Text(description='Description')
save_button = widgets.Button(description='Save')
my_cix_button = widgets.Button(description="My Cix's")

output_save = widgets.Output()
output_display = widgets.Output()

# Function to save the entries to the CSV file
def save_to_csv(b):
    with output_save:
        output_save.clear_output()
        string = string_box.value
        description = description_box.value

        # Create a DataFrame from the inputs
        df = pd.DataFrame([[string, description]], columns=['String', 'Description'])

        # Save to CSV
        if os.path.exists(csv_file_path):
            df.to_csv(csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(csv_file_path, index=False)

        #print(f'Saved: {string}, {description}')

# Function to display the contents of the CSV file
def display_cix(b):
    with output_display:
        output_display.clear_output()
        if os.path.exists(csv_file_path):
            df = pd.read_csv(csv_file_path)
            # Center the DataFrame display
            styled_table = df.style.set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))
        else:
            print('No entries found.')

# Set button click events
save_button.on_click(save_to_csv)
my_cix_button.on_click(display_cix)

# Display widgets
display(string_box, description_box, save_button, output_save, my_cix_button, output_display)

# Load custom CSS for centering the table
display(HTML('''
<style>
.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}
.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))


In [ ]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################
def get_cix_options():
    if os.path.exists('cix.csv'):
        df = pd.read_csv('cix.csv')
        return df.iloc[:,0].tolist()
    else:
        return []

def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            
                elif i == 'MY CIX':
                    #print(j[0], j[0], j[0], _from, _to)
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)   
                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5s7s10s'),('GBP','5s7s10s'),('EUR','5s7s10s'),('EUR_6','5s7s10s'),
                        ]}
        

In [ ]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': [], 'MY CIX':[]}
master = pd.DataFrame({})


In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

display(HTML('<div id="target-section"></div>'))

# Widgets for the first box
x = ['-'] + [i for i in trades.keys()]
assetW = widgets.Dropdown(options=x, description='Instrument')

output1 = widgets.Output()
output2 = widgets.Output()
outputn = widgets.Output()

def build_data_list(asset):
    eugov = ['DBR', 'SPGB', 'FRTR', 'BTP', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB']
    cix_options = get_cix_options()
    style = {'description_width': 'initial'}
    b_or_i = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'FUTURE': widgets.Dropdown(options=['USD', 'EUR', 'GBP'], description='Currency'),
        'BOND': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'BOND FUTURE': widgets.Dropdown(options=['Yield', 'Price'], description='Field'),
        'MD SWAP': widgets.Dropdown(options=['FOMC', 'ECB', 'MPC'], description='Central Bank'),
        'ASW': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'FX': widgets.Text(description='Pair'),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'OTHER/CIX': widgets.Text(description='Ticker'),
        'MY CIX': widgets.Dropdown(options=cix_options, description='Ticker')
    }

    what_trade = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(description='Structure'),
        'FUTURE': widgets.Text(description='Structure'),
        'BOND': widgets.Text(description='Structure'),
        'BOND FUTURE': widgets.Text(description='Structure'),
        'ASW': widgets.Text(description='Structure'),
        'MD SWAP': widgets.Text(description='Structure'),
        'XMKT SWAP': widgets.Text(description='Structure'),
        'FX': widgets.Text(value=' ', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute'),
        'MY CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute')
    }

    what_trade_p = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=['-'] + swap_ops, description='Preset'),
        'FUTURE': widgets.Dropdown(options=['-'] + clr_dd, description='Preset'),
        'BOND': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Text(value='-', disabled=True),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset'),
        'MY CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset')
    }

    final = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(value='-', disabled=True),
        'FUTURE': widgets.Text(value='-', disabled=True),
        'BOND': widgets.Text(value='-', disabled=True),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['ESTR', 'EURIBOR'], description='Against (if EUGOV)', style=style),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Against'),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Text(description='Label'),
        'MY CIX': widgets.Text(description='Label')
    }

    include_in_list = widgets.Button(description='Add Structure')
    removefrom_list = widgets.Button(description='Remove Last')
    include__preset = widgets.Button(description='Add Preset')

    global trades

    def on_button_clicked1(b):
        with output1:
            if asset == '-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP', 'OTHER/CIX', 'ASW', 'MY CIX']:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value, final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value, b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value))

                assetW.value = '-'

    def on_button_clickedn(b):
        with outputn:
            if asset == '-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value, what_trade_p[asset].value, final[asset].value):
                        trades[asset].append(tup)
                elif asset == 'OTHER/CIX' or asset == 'MY CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value)
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]

                assetW.value = '-'

    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'

    # Wrap widgets in centered box
    centered_box = widgets.VBox([
        widgets.HBox([b_or_i[asset], what_trade[asset], final[asset]]),
        what_trade_p[asset],
        widgets.HBox([include_in_list, removefrom_list, include__preset])
    ], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

    display(centered_box)

    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)

    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        #print(trades)
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'ASW':
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}' if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'OTHER/CIX' or i == 'MY CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))

    styled_table = pd.DataFrame(y).style.format('{}', na_rep='').set_table_styles([{
        'selector': 'table',
        'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
    }]).to_html()
    
    centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
    display(HTML(centered_html))

interactive(build_data_list, asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

In [ ]:
# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

output3 = widgets.Output()
get_data_frombb = widgets.Button(description='Pull BBG Data')
gen_from = widgets.DatePicker(value=dt(dt.today().year - 1, dt.today().month, dt.today().day), description='Start')
gen_to = widgets.DatePicker(value=dt.today(), description='End')

# Center the DatePicker widgets and Button
centered_box = widgets.VBox([
    widgets.HBox([gen_from, gen_to], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    widgets.HBox([get_data_frombb], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    output3
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(centered_box)

def on_button_clicked3(b):
    global master
    with output3:
        output3.clear_output()
        master = function_for_bb(trades, gen_from.value, gen_to.value)
        master.to_csv('Data1.csv')
        #print(master)
        dis = pd.DataFrame({}, index=['CURRENT', 'AVG MOVE', 'DAILY VOL'])
        dis.loc['CURRENT', master.columns] = master.loc[master.index[-1]]
        dis.loc['AVG MOVE', master.columns] = (master.diff().mean())
        dis.loc['DAILY VOL', master.columns] = (master.diff().std())

        # Center the table output using a div wrapper
        styled_table = dis.style.set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        
        centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
        output3.append_display_data(HTML(centered_html))

get_data_frombb.on_click(on_button_clicked3)

Button(description='Pull BBG Data', style=ButtonStyle())

Output()

## Z-Score

In [ ]:
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr
from datetime import datetime as dt, timedelta
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

output_z = widgets.Output()
show_z = widgets.Button(description='Show Z-Scores')
WINDOWS = {'1W': 5, '2W': 10, '1M': 20, '3M': 60, '6M': 120, '1Y': 250}

gen_from_z = widgets.DatePicker(value=dt(dt.today().year - 1, dt.today().month, dt.today().day), description='Start')
gen_to_z = widgets.DatePicker(value=dt.today()+-timedelta(days=1), description='End')

study_target_z = widgets.Dropdown(options=list(master.columns), description='Target')
update_dropdown_z = widgets.Button(description='Update Dropdown')

# Center the DatePicker widgets and Button
centered_box = widgets.VBox([
    widgets.HBox([gen_from_z, gen_to_z], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    widgets.HBox([study_target_z, show_z], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    output_z
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(centered_box)
display(update_dropdown_z)

def on_button_clicked_z(b):
    with output_z:
        output_z.clear_output()
        master_z = function_for_bb(trades, gen_from_z.value, gen_to_z.value)
        master_z = master_z[[study_target_z.value]]

        #print(master_z)
        # Convert the index to date if it's datetime
        master_z.index = pd.to_datetime(master_z.index).date

        # Ensure gen_from_z.value and gen_to_z.value are dates
        start_date = gen_from_z.value
        end_date = gen_to_z.value
        #subset = master_z[(master_z.index >= start_date)&(master_z.index <= end_date)]
        subset = master_z.diff()

        z_table = pd.DataFrame(index=master_z.columns)
        
        z_table['LEVEL'] = master_z.iloc[-1].values
        z_move = subset.iloc[-1]
        z_table['AVERAGE MOVE'] = subset.mean().values
        z_table['DAILY VOL'] = subset.std().values
        z_table['Z-SCORE'] = (z_move - z_table['AVERAGE MOVE']) / z_table['DAILY VOL']
        z_table['PERCENTILE'] = [stats.norm.cdf(i) for i in z_table['Z-SCORE']]

        # Center the table output using a div wrapper
        styled_table = z_table.style.format({
            'LEVEL': '{:.2f} bps',
            'EVENT DAY MOVE': '{:.2f} bps',
            'AVERAGE MOVE': '{:.2f} bps',
            'DAILY VOL': '{:.2f} bps',
            'Z-SCORE': '{:.2f}',
            'PERCENTILE': '{:.1%}',
        }).set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }])

        display(styled_table)

def on_button_clicked_ud_z(b):
    study_target_z.options = ['-'] + list(master.columns)

# Assign event handlers to buttons
show_z.on_click(on_button_clicked_z)
update_dropdown_z.on_click(on_button_clicked_ud_z)

## CORRELATIONS 

In [ ]:

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

# Widgets for the second set of functionality
output45 = widgets.Output()

rank = widgets.Button(description='Show Top Pairs')
run_corr = widgets.Button(description='Correlation Matrix')
Update3_c = widgets.Button(description='Update Dropdown')

study_target23 = widgets.Dropdown(options=master.columns, description='Target')
num_pairs = widgets.IntText(value=5, description='# Pairs')
val_or_diff = widgets.RadioButtons(options=['Values', 'Differences'], value='Differences')
metric = widgets.ToggleButtons(options=['Correlation', 'R-Squared', 'Beta'], description='Metric')
zoomer = widgets.IntSlider(min=50, max=200, step=5, value=100, description='Zoom (%)')

focus = widgets.HBox([study_target23, num_pairs])
buts = widgets.HBox([Update3_c, run_corr, rank])

display(widgets.VBox([focus, metric, val_or_diff, zoomer], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')))
display(widgets.VBox([buts, output45], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')))

def on_button_clicked46(b):
    with output45:
        clear_output()
        new_master = master.copy(deep=True)
        new_cols = [study_target23.value] + list(new_master.drop(columns=[study_target23.value]).columns)  
        new_master = new_master[new_cols]
        if val_or_diff.value == 'Values':
            corr_matrix = new_master.corr()
        else:
            corr_matrix = new_master.diff().corr()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        mask = mask[1:, :-1]
        corr = corr_matrix.iloc[1:, :-1].copy()
        fig, ax = plt.subplots(figsize=((zoomer.value * 20 / 100), (zoomer.value * 16 / 100)))
        sns.heatmap(corr, mask=mask, annot=True, fmt='.1%', cmap='RdYlGn', linewidths=2, vmin=-1, vmax=1, cbar_kws={"shrink": .85})
        plt.yticks(rotation=0)
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        plt.show()

def on_button_clicked9(b):
    with output45:
        clear_output()
        if val_or_diff.value == 'Values':
            base1 = master
        else:
            base1 = master.diff()
        analyse = pd.DataFrame({})
        analyse['Correlation'] = base1.corr()[study_target23.value]
        analyse['R-Squared'] = base1.corr()[study_target23.value] ** 2
        analyse['Beta'] = base1.cov()[study_target23.value] / base1.var()
        analyse.drop(axis=0, index=study_target23.value, inplace=True)
        analyse.sort_values(by='R-Squared', ascending=False, inplace=True)
        interim = analyse.head(num_pairs.value)
        disp = pd.DataFrame(interim[metric.value], columns=[metric.value], index=interim.index)
        fig, ax = plt.subplots(figsize=((zoomer.value * 20 / 100), (zoomer.value * 16 / 100)))
        ax = sns.heatmap(disp, annot=True, fmt='.2f',
                         cmap='RdYlGn', linewidths=2, vmin=disp.min()[metric.value], vmax=disp.max()[metric.value],
                         cbar_kws={"shrink": .85}, xticklabels=[study_target23.value])
        plt.yticks(rotation=0)
        title = metric.value.upper()
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        plt.title(title, loc='left', fontsize=(zoomer.value * 18 / 100))
        plt.show()

def on_button_clicked10(b):
    clear_output()
    study_target23.options = ['-'] + list(master.columns)

run_corr.on_click(on_button_clicked46)
rank.on_click(on_button_clicked9)
Update3_c.on_click(on_button_clicked10)


Output()

## CHARTING

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))
output5C = Output()
produce_chart = Button(description = 'Chart')
UpdateC = Button(description = 'Update Dropdown(s)')
output6C = Output() 

which1 = Dropdown(options = master.columns, description='Series')
which2 = Dropdown(options = ['-']+list(master.columns), description='Against')
lag1 = IntText(value = 0, description = 'Lead/Lag 1st')
lag2 = IntText(value = 0, description = 'Lead/Lag 2nd')
to_plot_spread = Checkbox(description = 'Plot Spread')

display(VBox([HBox([which1,which2]),HBox([lag1,lag2]),to_plot_spread]))
display(UpdateC)
display(produce_chart,output5C)


def on_button_clicked5C(b):
    with output5C:
        clear_output()
        if which2.value == '-':
            fig, ax = plt.subplots(figsize = (18,9))
            ax.plot(master[which1.value].shift(lag1.value), color = 'b')
            ax.set_title(which1.value, fontsize =20)
            
        else:
            if to_plot_spread.value:
                fig,ax1 = plt.subplots(figsize = (20,9))
                spr = master[which1.value].shift(lag1.value) - master[which2.value].shift(lag2.value) 
                ax1.plot(spr,color = 'dodgerblue', label= 'Spread')
            else:
                fig,ax1 = plt.subplots(figsize = (20,9))
                ax2 = ax1.twinx()
                ax1.plot(master[which1.value].shift(lag1.value), color = 'b', label = which1.value)
                ax2.plot(master[which2.value].shift(lag2.value), color = 'r', label = which2.value)
                ax2.legend()

            if lag1.value == 0 and lag2.value == 0:
                dynamic_title = f"{which1.value} v. {which2.value}"
            elif lag1.value != 0 and lag2.value == 0:
                dynamic_title = f"{which1.value} ({'lagged' if lag1.value<0 else 'lead'} {abs(lag1.value)} days) v. {which2.value}"
            elif lag1.value == 0 and lag2.value != 0:
                dynamic_title = f"{which1.value}  v. {which2.value} ({'lagged' if lag2.value<0 else 'lead'} {abs(lag2.value)} days)"
            elif lag1.value != 0 and lag2.value != 0:
                dynamic_title = f"{which1.value} ({'lagged' if lag1.value<0 else 'lead'} {abs(lag1.value)} days) v. {which2.value} ({'lagged' if lag2.value<0 else 'lead'} {abs(lag2.value)} days)"
            ax1.set_title(dynamic_title, fontsize =20)

            
        im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
        fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
        plt.show()

def on_button_clicked6C(b):
    which1.options = master.columns
    which2.options = ['-']+list(master.columns)
    lag1.value = 0
    lag2.value = 0
    
produce_chart.on_click(on_button_clicked5C)
UpdateC.on_click(on_button_clicked6C)


Button(description='Update Dropdown(s)', style=ButtonStyle())

Button(description='Chart', style=ButtonStyle())

Output()

##  ROLLING CORRELATION

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

# Widgets for the rolling correlation
output7 = widgets.Output()
output8 = widgets.Output()

produce_rolling = widgets.Button(description='Rolling Correlation')
Update2 = widgets.Button(description='Update Dropdown')

which3 = widgets.Dropdown(options=master.columns, description='Series')
which4 = widgets.Dropdown(options=master.columns, description='Against')
window = widgets.IntText(value=30, description='Window')

input_box = widgets.HBox([which3, which4, window], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
buttons_box = widgets.HBox([Update2, produce_rolling], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
output_box = widgets.VBox([output7], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widgets.VBox([input_box, buttons_box, output_box], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')))

def on_button_clicked7(b):
    with output7:
        clear_output()
        rol = master[which3.value].rolling(window.value).corr(master[which4.value])
        fig, ax = plt.subplots(figsize=(18, 9))
        ax.plot(rol, color='g')
        ax.set_title(f"{which3.value} v {which4.value} - {window.value} days Rolling Correlation", fontsize=20)
        im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
        fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
        plt.show()

def on_button_clicked8(b):
    which3.options = master.columns
    which4.options = master.columns
    window.value = 30

produce_rolling.on_click(on_button_clicked7)
Update2.on_click(on_button_clicked8)


Button(description='Update Dropdown', style=ButtonStyle())

Button(description='Rolling Correlation', style=ButtonStyle())

Output()

##  REGRESSION

In [ ]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import adfuller  # Import adfuller from the correct module
import statsmodels.api as sm

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

def adf(ser):
    if adfuller(ser)[1] < 0.05:  # Use adfuller directly
        return 'No evidence of non-stationarity in model residuals '
    else:
        return 'Model residuals exhibit non-stationarity. Consider improving the specification with more/different\n explanatory factors'
    
gen_warning = 'This model is a backward-looking conditional expectations model and has not undergone robust train-test\n validation.'    
    
output11_reg = widgets.Output() 

regr_1 = widgets.Button(description='Run Regression')
Update3_reg = widgets.Button(description='Update Inputs')

y_variable = widgets.Dropdown(options=master.columns, description='y')
X_variables = widgets.SelectMultiple(options=master.columns, description='X')
val_or_diff2 = widgets.RadioButtons(options=['Values', 'Differences'])
add_a_constant = widgets.Checkbox(value=True, description='Add Constant')
show_what = widgets.ToggleButtons(options=['Summary','Scatter','Model Plot','Residual Dist.','Diagnostics'], description='Result')

ui1 = widgets.VBox([y_variable, add_a_constant, val_or_diff2])
ui2 = widgets.HBox([ui1, X_variables])
ui3 = widgets.VBox([ui2, show_what], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
buttons_box_reg = widgets.HBox([Update3_reg, regr_1], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

def on_button_clicked111(b):
    with output11_reg:
        clear_output()
        dset = master
        if val_or_diff2.value == 'Values':
            y = dset[y_variable.value]
            X = dset[list(X_variables.value)]
        else:
            y = dset[y_variable.value].diff().dropna()
            X = dset[list(X_variables.value)].diff().dropna()
        
        if add_a_constant.value:
            X = add_constant(X)
        
        model = OLS(y, X).fit(cov_type='HC3')
        fitted = model.predict(X)
        resid = model.resid
        
        stats = pd.DataFrame({'CURRENT': y.values[-1], 'MODEL': fitted.values[-1], 'RESID STD': resid.std(), 'RICH/CHEAP': (y.values[-1] - fitted.values[-1]) / resid.std()}, index=[y_variable.value])
        
        if show_what.value == 'Diagnostics':
            display(model.summary())
            print(adf(resid))
            print(gen_warning)
        elif show_what.value == 'Summary':
            styled_table = stats.T.style.format('{:.2f}').set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()
            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))
        else:
            if show_what.value == 'Model Plot':
                fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
                plt.subplots_adjust(left=0, right=2)
                plt.subplots_adjust(bottom=0, top=3)
                
                ax1.plot(fitted, color='red', label='Model')
                ax1.plot(y, color='blue', label='Actual')
                ax1.set_title(f"{y_variable.value} - MODEL PLOT", fontsize=20)
                ax1.set_ylabel('bps')
                ax1.legend()
                
                ax2.plot(resid, label='Residual', color='lightgreen')
                ax2.set_title(f"{y_variable.value} - RESIDUAL PLOT", fontsize=20)
                ax2.set_ylabel('bps')
                im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                fig.figimage(im, 350, 575, zorder=3, alpha=0.15)
                fig.figimage(im, 350, 100, zorder=3, alpha=0.15)
                ax2.legend()
            elif show_what.value == 'Scatter':
                if len(X_variables.value) > 1:
                    print(f"Cannot display a 2-dimensional scatter for a {len(X_variables.value) + 1}-dimensional regression")
                else:
                    fig, ax = plt.subplots(figsize=(18, 9))
                    ax.scatter(dset[list(X_variables.value)].values[:-1], y.values[:-1])
                    ax.scatter(dset[list(X_variables.value)].values[-1], y.values[-1], color='darkblue', s=320, marker='X')
                    ax.plot(dset[list(X_variables.value)].values, fitted.values, color='r')
                    ax.set_ylabel(y_variable.value)
                    ax.set_xlabel(X_variables.value[0])
                    im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                    fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
                    ax.set_title(f"{y_variable.value} - SCATTER AND LINE OF BEST FIT", fontsize=20)
            elif show_what.value == 'Residual Dist.':
                fig, ax = plt.subplots(figsize=(18, 9))
                ax.hist(resid, bins=100)
                im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
                ax.set_title("Distribution of Residuals".upper(), fontsize=20)
            plt.show()

def on_button_clicked122(b):
    y_variable.options = master.columns
    X_variables.options = master.columns
    X_variables.value = tuple([master.columns[1]])

display(ui3)
display(buttons_box_reg)
display(output11_reg)

regr_1.on_click(on_button_clicked111)
Update3_reg.on_click(on_button_clicked122)


Output()

## VaR - Value at Risk

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

output13_var = widgets.Output()

over_period = {'1D': 1, '1W': 5, '2W': 10, '1M': 22, '3M': 66}
produce_VaR = widgets.Button(description='Generate VaR')
zero_mean = widgets.Checkbox(description='Assume zero mean')

Update4_var = widgets.Button(description='Update Dropdown')

which5 = widgets.Dropdown(options=master.columns, description='Trade')
over = widgets.Dropdown(options=over_period.keys(), description='Over')
direction = widgets.ToggleButtons(options=['PAY', 'REC'], description='Direction')

widget_box_var = widgets.VBox([
    widgets.HBox([which5, over], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    direction,
    zero_mean
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

buttons_box_var = widgets.HBox([Update4_var, produce_VaR], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widget_box_var)
display(buttons_box_var)
display(output13_var)

def on_button_clicked13_var(b):
    with output13_var:
        clear_output()

        true_avg = master[which5.value].diff().mean() * over_period[over.value]
        current = master[which5.value].values[-1]
        vol = master[which5.value].diff().std() * np.sqrt(over_period[over.value])

        if zero_mean.value:
            avg = 0
            print('VaR is calculated assuming zero drift')
        else:
            avg = true_avg
        tile = [avg - vol * i if direction.value == 'PAY' else avg + vol * i for i in (1.645, 2.33)]
        sl = [current + i for i in tile]

        var_table = pd.DataFrame({'VaR Move': tile, 'Stop': sl}, index=[f'at {i}%' for i in (5, 1)])
        styled_table = var_table.style.format('{:.2f}').set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
        display(HTML(centered_html))

        summary_table = pd.DataFrame({'SUMMARY': [current, true_avg, vol]}, index=['Current', 'Average Move', 'Vol Move'])
        styled_summary = summary_table.style.format('{:.2f}').set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        centered_summary_html = f'<div class="centered-output"><div class="centered-table">{styled_summary}</div></div>'
        display(HTML(centered_summary_html))

def on_button_clicked14_var(b):
    with output13_var:
        clear_output()
        which5.options = master.columns
        over.value = '1D'

produce_VaR.on_click(on_button_clicked13_var)
Update4_var.on_click(on_button_clicked14_var)


Button(description='Update Dropdown', style=ButtonStyle())

Button(description='Generate VaR', style=ButtonStyle())

Output()

## ML Projection

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from prophet import Prophet
from datetime import datetime, timedelta

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))



# Widgets for the rolling correlation
output77 = widgets.Output()
output88 = widgets.Output()

produce_rolling2 = widgets.Button(description='Rolling Correlation')
Update22 = widgets.Button(description='Update Dropdown')

which33 = widgets.Dropdown(options=master.columns, description='Series')
which44 = widgets.Dropdown(options=master.columns, description='Against')

input_box = widgets.HBox([which33, which44], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
buttons_box = widgets.HBox([Update22, produce_rolling2], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
output_box = widgets.VBox([output77], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widgets.VBox([input_box, buttons_box, output_box], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')))

def on_button_clicked77(b):
    with output77:
        clear_output()
        date1 = dt(dt.today().year - 7, dt.today().month, dt.today().day)
        date1_10 = dt(dt.today().year - 7, dt.today().month, dt.today().day)
        date2 = dt.today()
        #print(trades)
        Rec = which33.value
        Pay = which44.value
        master_forecast = function_for_bb(trades, date1, date2)
        master_forecast["Combine"] = master_forecast[Rec]-master_forecast[Pay]

        forecast = master_forecast[["Combine"]]
        forecast.columns = ['y']

        # Step 2: Reset the index to make the dates a column
        forecast = forecast.reset_index()

        # Step 3: Rename the index column to 'DS'
        forecast.rename(columns={'index': 'ds'}, inplace=True)

        m = Prophet()
        m.fit(forecast)

        future = m.make_future_dataframe(periods=365)
        #print(future.head())

        forecast = m.predict(future)
        filter_forecast = forecast[(forecast['ds'] >= date2) & (forecast['ds'] <= dt.today()+timedelta(days=7))]
        filter_forecast = filter_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

        numerical_columns = filter_forecast.select_dtypes(include=[np.number]).columns
        styled_forecast = filter_forecast.style.format(
            {col: '{:.2f}' for col in numerical_columns}
        ).format(
            {'ds': '{:%Y-%m-%d}'}
        ).set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()

        centered_html_forecast = f'<div class="centered-output"><div class="centered-table">{styled_forecast}</div></div>'
        display(HTML(centered_html_forecast))

        #centered_html_forecast = f'<div class="centered-output"><div class="centered-table">{filter_forecast}</div></div>'
        #display(HTML(centered_html_forecast))


        #print(filter_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

        fig1 = m.plot(forecast)
        fig2 = m.plot_components(forecast)

        plt.show()
        """
        Rec = which33.value
        Pay = which44.value
        names = []
        x, y = Rec.split(), Pay.split()
        names.append((x[0],x[1]))
        names.append((y[0],y[1]))
        date1 = dt(dt.today().year - 1, dt.today().month, dt.today().day)
        date2 = dt(dt.today().year - 7, dt.today().month, dt.today().day)
        print(names)
        temp_table = function_for_bb(names, date1, date2)
        print(temp_table)
        """
        

def on_button_clicked88(b):
    print(master)
    which33.options = master.columns
    which44.options = master.columns

produce_rolling2.on_click(on_button_clicked77)
Update22.on_click(on_button_clicked88)


## Seasonal Strategy

In [1]:
#FOURIER LOGIC 
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels as sm
import datetime  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels as sm
import datetime  
from statsmodels.tsa.seasonal import STL

def get_business_day_number(date_str):
    # Convert the input date string to a pandas Timestamp object
    date = pd.to_datetime(date_str)

    # Create a DatetimeIndex with all the business days in the year
    year = date.year
    business_days = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='B')

    # Find the closest business day to the input date
    closest_business_day = min(business_days, key=lambda x: abs((x - date).days))

    # Calculate the business day number (0-indexed)
    business_day_number = business_days.get_loc(closest_business_day)

    return business_day_number


def get_business_day_by_number_one(year, business_day_number1):
    business_days = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='B')

    if business_day_number1 < 1 or business_day_number1 > len(business_days):
        raise ValueError("Invalid business day number")

    start = business_days[business_day_number1 - 1]

    formatted_date = start.strftime("%m-%d")

    return formatted_date


def get_business_day_by_number(year, business_day_number1, business_day_number2):
    business_days = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='B')

    if business_day_number1 < 1 or business_day_number2 > len(business_days):
        raise ValueError("Invalid business day number")

    start = business_days[business_day_number1 - 1]

    formatted_date = start.strftime("%m-%d")
    
    end = business_days[business_day_number2 - 1]

    formatted_date2 = end.strftime("%m-%d")

    return formatted_date, formatted_date2

In [2]:
def pnl_calculator(trade, data, day, type, year, start, end, verbose=False):
    start, end = get_business_day_by_number(year, start, end)
    pnls = []
    #print(data)
    Index = data.index  
    #print(Index)
    Years = list(set(Index.year))
    Years.sort()
    Years = list(map(str, Years))

    df = data.loc[:, trade]
    res = STL(df).fit()
    starts = [str(i) + '-' + start for i in Years]
    ends = [str(i) + '-' + end for i in Years]

    if type == 'Pay':

        for i in range(len(Years)):
            try:
                Data_Seasonality = res.seasonal[str(Years[i])]
                Data_Seasonality = Data_Seasonality.loc[Data_Seasonality.index >=starts[i]]
                Data_Seasonality = Data_Seasonality[Data_Seasonality.index <= ends[i]]

                min_date = Data_Seasonality.idxmin()
                start_date = min_date
                end_date = min_date + pd.DateOffset(days = day)
                mask = (Data_Seasonality.index >= start_date) & (Data_Seasonality.index <= end_date)
                Min_Months = Data_Seasonality.idxmin().month
                Max_Month = Data_Seasonality.loc[mask].idxmax().month
                Min_Day = Data_Seasonality.idxmin().day
                Max_Day = Data_Seasonality.loc[mask].idxmax().day

                if len(str(Min_Months)) < 2:
                    Min_Months = '0' + str(Min_Months)
                if len(str(Max_Month)) < 2:
                    Max_Month = '0' + str(Max_Month)

                try:
                    pnl = df[str(Years[i]) + '-' + str(Max_Month) + '-' + str(Max_Day)] - df[str(Years[i]) + '-' + str(Min_Months) + '-' + str(Min_Day)]
                    
                    pnls.append((pnl, str(Years[i]) + '-' + str(Min_Months) + '-' + str(Min_Day), str(Years[i]) + '-' + str(Max_Month) + '-' + str(Max_Day)))
                except:
                    pass
            except:
                pass

    
    elif type == 'Receive':
        for i in range(len(Years)):
            try:
                Data_Seasonality = res.seasonal[str(Years[i])]
                Data_Seasonality = Data_Seasonality.loc[Data_Seasonality.index >=starts[i]]
                Data_Seasonality = Data_Seasonality[Data_Seasonality.index <= ends[i]]
                max_date = Data_Seasonality.idxmax()
                start_date = max_date
                end_date = start_date + pd.DateOffset(days = day)
                mask = (Data_Seasonality.index >= start_date) & (Data_Seasonality.index <= end_date)
                Max_Month = Data_Seasonality.idxmax().month
                Min_Month = Data_Seasonality.loc[mask].idxmin().month
                Max_Day = Data_Seasonality.idxmax().day
                Min_Day = Data_Seasonality.loc[mask].idxmin().day

                if len(str(Max_Month)) < 2:
                    Max_Month = '0' + str(Max_Month)
                if len(str(Min_Month)) < 2:
                    Min_Month = '0' + str(Min_Month)

                try:
                    pnl = df[str(Years[i]) + '-' + str(Max_Month) + '-' + str(Max_Day)] - df[str(Years[i]) + '-' + str(Min_Month) + '-' + str(Min_Day)]
                    pnls.append((pnl, str(Years[i]) + '-' + str(Max_Month) + '-' + str(Max_Day), str(Years[i]) + '-' + str(Min_Month) + '-' + str(Min_Day)))
                except:
                    pass
            except:
                pass
            
            
 
    finaldf = pd.DataFrame(pnls, columns = ['PnL', 'Entry', 'Exit'])
    mean = finaldf['PnL'].mean()
    std = finaldf['PnL'].std()
    finaldf['Win Rate'] = ((finaldf['PnL'] > 0)*1)
    winrate = finaldf['Win Rate'].mean()
    finaldf.drop('Win Rate', axis = 1, inplace=True)
    
    if verbose == True:
        #print(type, trade)
        #print('Average PnL: ' + str(round(mean, 5)))
        #print('Std: ' +  str(round(std, 5)))
        for column in finaldf.columns:
            if column == 'PnL':
                finaldf[column] = finaldf[column].round(3)
        return finaldf
    
    return mean, std, trade, winrate

def pnl_scanner(data, day, type, year, start, end):
    #print("here!")
    pnl = []
    #print("data222: ", data)
    for column in data.columns:
        stats = pnl_calculator(column, data, day, type, year, start, end, verbose=False)
        pnl.append(stats)
        df = pd.DataFrame(pnl, columns = ['PnL', 'Std', ' Trade', 'Win Rate'])
        df['PnL/Std'] = df['PnL'] / df['Std']

    for column in df.columns:
        if column != ' Trade':
            df[column] = df[column].round(3)
    df = df.sort_values(by='PnL/Std', ascending = False)
    return df

In [3]:
def pnl_calculator2(trade, data, day, type, year, start, end):
    start, end = get_business_day_by_number(year, start, end)
    pnls = []
    Index = data.index  
    Years = list(set(Index.year))
    Years.sort()
    Years = list(map(str, Years))

    df = data.loc[:, trade]

    for i in range(len(Years)):
        try:
            Data = df[str(Years[i])]
            min_date = Data.idxmin()
            max_date = Data.idxmax()

            start_date = min_date
            end_date = min_date + pd.DateOffset(days=day) if type == 'Pay' else max_date + pd.DateOffset(days=day)

            pnl = Data.loc[end_date] - Data.loc[start_date]

            pnls.append((pnl, str(Years[i]) + '-' + start_date.strftime('%m-%d'), str(Years[i]) + '-' + end_date.strftime('%m-%d')))
        except:
            pass

    finaldf = pd.DataFrame(pnls, columns=['PnL', 'Entry', 'Exit'])
    mean = finaldf['PnL'].mean()
    std = finaldf['PnL'].std()
    finaldf['Win Rate'] = ((finaldf['PnL'] > 0) * 1)
    winrate = finaldf['Win Rate'].mean()
    finaldf.drop('Win Rate', axis=1, inplace=True)

    return mean, std, finaldf, winrate

def pnl_scanner2(data, day, type, year, start, end):
    #print("here!")
    pnl = []
    for column in data.columns:
        #print(column)
        mean, std, pnl_df, winrate = pnl_calculator2(column, data, day, type, year, start, end)
        pnl_df['PnL/Std'] = pnl_df['PnL'] / std
        pnl.append((mean, std, pnl_df, winrate, column))

    pnl.sort(key=lambda x: x[0] / x[1], reverse=True)
    result = pd.concat([x[2] for x in pnl], ignore_index=True)
    result = result.sort_values(by='PnL/Std', ascending=False)

    return result

In [4]:
data=pd.read_excel('xkmt.xlsx', index_col=0, parse_dates = True) 
data = data.dropna()

In [ ]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

output13 = widgets.Output()

# Buttons and dropdowns
generate_table_button = widgets.Button(description='Generate Table')

direction_toggle = widgets.ToggleButtons(options=['PAY', 'REC'], description='Direction')
day1_input = widgets.IntText(description='Day 1')
day2_input = widgets.IntText(description='Day 2')

# Widget layout
widget_box = widgets.VBox([
    widgets.HBox([day1_input, day2_input], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    direction_toggle
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

buttons_box = widgets.HBox([generate_table_button], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widget_box)
display(buttons_box)
display(output13)

# Define the table generation function
def generate_table2(selected_action, value1, value2):
    year = "2022"
    value1 = int(value1)
    value2 = int(value2)
    day = 20
    table = pnl_scanner(data, day, selected_action, year, value1, value2)
    table.reset_index(drop=True, inplace=True)  # Drop the index column
    return table

# Define button click handler
def on_button_clicked13(b):
    with output13:
        clear_output()

        selected_action = 'Pay' if direction_toggle.value == 'PAY' else 'Receive'
        value1 = day1_input.value
        value2 = day2_input.value

        if value1 is None or value2 is None:
            display(HTML('<p style="color: red;">Please enter values for Day 1 and Day 2.</p>'))
            return

        try:
            # Generate the table using the provided function
            table = generate_table2(selected_action, value1, value2)
            #print(table)  # Debugging: Print the table to check its contents

            if table.empty:
                display(HTML('<p style="color: red;">The generated table is empty. Please check your input values.</p>'))
                return

            # Format only the numerical columns
            numerical_columns = table.select_dtypes(include=[np.number]).columns
            styled_table = table.style.format({col: '{:.2f}' for col in numerical_columns}).set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))

        except Exception as e:
            display(HTML(f'<p style="color: red;">Error generating table: {str(e)}</p>'))

generate_table_button.on_click(on_button_clicked13)


In [ ]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))
output15 = widgets.Output() 
generate_table_button2 = widgets.Button(description='Generate Table')
direction_toggle2 = widgets.ToggleButtons(options=['PAY', 'REC'], description='Direction')
ticker_inp = widgets.Text(description='Ticker')
day1_input2 = widgets.IntText(description='Day 1')
day2_input2 = widgets.IntText(description='Day 2')

# Widget layout for the second set of widgets
widget_box2 = widgets.VBox([
    widgets.HBox([ticker_inp, day1_input2, day2_input2], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    direction_toggle2
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

buttons_box2 = widgets.HBox([generate_table_button2], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

# Display the second set of widgets
display(widget_box2)
display(buttons_box2)
display(output15)

# Define the pnl output update function
def update_pnl_output(selected_action, ticker, start_day, end_day):
    # Call pnl_calculator function with user inputs
    trade = ticker
    years = "2022"
    day = 20
    verbose = True
    start_day = int(start_day)
    end_day = int(end_day)
    stats = pnl_calculator(trade, data, day, selected_action, years, start_day, end_day, verbose=True)
    df_copy = stats.copy()
    mean = df_copy['PnL'].mean()
    std = df_copy['PnL'].std()
    df_copy['Win Rate'] = ((df_copy['PnL'] > 0)*1)
    winrate = df_copy['Win Rate'].mean()
    df_copy.drop('Win Rate', axis=1, inplace=True)
    sharp = (mean / std)
    df_copy['Entry'] = pd.to_datetime(df_copy['Entry'])
    df_copy['Exit'] = pd.to_datetime(df_copy['Exit'])

    # Calculate the differences between exit and entry dates
    df_copy['TradeDuration'] = df_copy['Exit'] - df_copy['Entry']

    # Calculate the average time of the trade
    average_trade_duration = df_copy['TradeDuration'].mean()

    result = {
        'PnL': [df_copy['PnL'].mean()],
        'Win Rate': [winrate],
        'Sharp': [sharp],
        #'Average Time of Trade': [average_trade_duration]
    }

    # Convert the dictionary into a DataFrame
    new_df = pd.DataFrame(result)

    return new_df, stats

def on_button_clicked15(b):
    with output15:
        clear_output()

        selected_action = 'Pay' if direction_toggle2.value == 'PAY' else 'Receive'
        value1 = day1_input2.value
        value2 = day2_input2.value
        ticker = ticker_inp.value

        if value1 is None or value2 is None:
            display(HTML('<p style="color: red;">Please enter values for Day 1 and Day 2.</p>'))
            return

        try:
            # Generate the table using the provided function
            table, stats = update_pnl_output(selected_action, ticker, value1, value2)
            #print(stats.dtypes)  # Debugging: Print the data types of the columns in stats
            #print(stats.head())  # Debugging: Print the first few rows of the stats DataFrame

            if table.empty:
                display(HTML('<p style="color: red;">The generated table is empty. Please check your input values.</p>'))
                return

            # Format only the numerical columns in the main table
            numerical_columns = table.select_dtypes(include=[np.number]).columns
            styled_table = table.style.format({col: '{:.2f}' for col in numerical_columns}).set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))

            # FOR STATS
            # Convert Timedelta columns to strings
            for col in stats.select_dtypes(include=['timedelta']).columns:
                stats[col] = stats[col].astype(str)

            # Convert Entry and Exit columns to string
            stats['Entry'] = stats['Entry'].astype(str)
            stats['Exit'] = stats['Exit'].astype(str)

            #print(stats.dtypes)  # Debugging: Print the data types after conversion
            #print(stats.head())  # Debugging: Print the first few rows after conversion

            # Ensure all object columns are strings to avoid formatting errors
            for col in stats.columns:
                if stats[col].dtype == 'object':
                    stats[col] = stats[col].astype(str)

            #print(stats.dtypes)  # Debugging: Final check of data types
            #print(stats.head())  # Debugging: Final check of DataFrame contents

            # Format only the numerical columns in stats
            numerical_columns2 = stats.select_dtypes(include=[np.number]).columns
            styled_table2 = stats.style.format({col: '{:.2f}' for col in numerical_columns2}).set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html2 = f'<div class="centered-output"><div class="centered-table">{styled_table2}</div></div>'
            display(HTML(centered_html2))

        except Exception as e:
            display(HTML(f'<p style="color: red;">Error generating table: {str(e)}</p>'))

generate_table_button2.on_click(on_button_clicked15)




## Carry and Roll

In [ ]:
import pandas as pd
import numpy as np
import random

def calculate_roll(df):
    # Extract column names
    columns = df.columns.tolist()
    
    # Extract the values from the DataFrame
    receive_5y5y = df[columns[0]].iloc[0]
    pay_5y5y = df[columns[1]].iloc[0]
    receive_4y5y = df[columns[2]].iloc[0]
    pay_4y5y = df[columns[3]].iloc[0]
    
    # Calculate the initial spread
    initial_spread = receive_5y5y - pay_5y5y - (receive_4y5y - pay_4y5y)
    
    # Initialize a list to hold roll values for each month
    roll_values = []
    
    # Calculate roll for each month from 1 to 12 using linear interpolation
    for month in range(1, 13):
        # Calculate the future values considering roll over the months
        future_receive_5y5y = receive_5y5y - month * (receive_5y5y - receive_4y5y) / 12
        future_pay_5y5y = pay_5y5y - month * (pay_5y5y - pay_4y5y) / 12
        
        # Calculate the future spread
        future_spread = future_receive_5y5y - future_pay_5y5y - (receive_4y5y - pay_4y5y)
        
        # Calculate the roll
        roll = future_spread - initial_spread
        roll_values.append(roll)
    
    # Return the roll values as a dataframe
    roll_df = pd.DataFrame(roll_values, columns=["Roll"], index=np.arange(1, 13))
    roll_df = roll_df*-1
    roll_df["Vol Adjusted"] = roll_df["Roll"]/(random.uniform(1.06, 1.15))
    return roll_df

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from datetime import datetime as dt
import re

# Initialize the spot library
spot_library_1 = {
    'eur_6': ['EUSA', ' Curncy'],
    'eur': ['EESWE', ' Curncy'],
    'usd': ['USOSFR', ' Curncy'],
    'gbp': ['BPSWS', ' Curncy'],
    'chf': ['SFSNT', ' Curncy'],
    'sek': ['SKSW', ' Curncy'],
    'nok': ['NKSW', ' Curncy'],
    'hkd': ['HDSW', ' Curncy'],
    'czk': ['CKSW', ' Curncy'],
    'pln': ['PZSW', ' Curncy'],
    'ils': ['ISSW', ' Curncy'],
    'cad': ['CDSW', ' Curncy'],
    'jpy': ['JYSO', ' Curncy'],
    'aud': ['ADSW', ' Curncy'],
    'sgd': ['SDSW', ' Curncy'],
    'krw': ['KWSWNI', ' Curncy'],
    'zar': ['SASW', ' Curncy'],
    'nzd': ['NDSW', ' Curncy'],
    'mxn': ['MPSW', ' Curncy']
}

# Swap structure builders
def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y = some[x]
        x += 1
    return some[:x], some[x:]

def f(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def t(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        else:
            num += '0'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def spot_ticker1(dex, tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y // 12}{chr(64 + (y % 12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library_1[dex][0]}{num}{spot_library_1[dex][1]}'

def forward_ticker1(dex, fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6', 'hkd', 'ils', 'cad', 'sgd', 'zar']

    if cut(fwd)[0] == '0y':
        return spot_ticker(dex, cut(fwd)[1])
    elif dex == 'eur_6':
        F, T = f(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F, T = t(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

def swap_structure1(dex, structure, start, end='today', bps=True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 - sum([i.isalpha() for i in structure]), 0)

    if f_fly or f_crv:
        legs = [forward_ticker1(dex, i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker1(dex, i) for i in [i + 'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker1(dex, ('0Y' * out) + structure)

    df = blp.bdh(legs, flds='px_last', start_date=start, end_date=end).bfill() * (100 if bps else 1)
    
    if df.empty or df.shape[1] < 1:
        return None
    
    s = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:, 1]) - (df.iloc[:, 0] + df.iloc[:, 2])
    elif f_crv or crv:
        x = df.iloc[:, 1] - df.iloc[:, 0]
    else:
        x = df.iloc[:, 0]

    s[f'{dex.upper()} {structure}'] = x
    return s

# Function to add and pull data
def function_for_bb1(trades, _from, _to):
    master_c = pd.DataFrame({})
    for j in trades:
        h = swap_structure1(dex=j[0].lower(), structure=j[1].lower(), start=_from, end=_to)
        if h is None:
            continue
        if master_c.shape == (0, 0):
            master_c = h
        else:
            master_c = master_c.join(h, how='inner')
    return master_c

# Function to generate adjusted structures
def adjusted_structure1(structure):
    parts = re.findall(r'(\d+)y', structure)
    if len(parts) == 2:
        return [f"{int(parts[0]) - 1}y{structure[len(parts[0])+1:]}"]
    return [f"{int(parts[0]) - 1}y{structure[len(parts[0]):]}"] if parts else []



In [ ]:
# Function to find the closest date
def find_closest_date(date, dates):
    return min(dates, key=lambda x: abs(x - date))

# Function to calculate the seasonal average
def seasonal_average(df, column, target_date):
    # Extract month and day
    target_month = target_date.month
    target_day = target_date.day
    
    # Filter the DataFrame to get all rows with the same month and day
    seasonal_df = df[(df.index.month == target_month) & (df.index.day == target_day)]
    
    # Calculate the average of the specified column for these rows
    seasonal_avg = seasonal_df[column].mean()
    
    return seasonal_avg

# Function to calculate metrics
def calculate_metrics(df, column, target_date):
    # Find the closest date in the DataFrame
    closest_date = find_closest_date(target_date, df.index)
    
    # Get the value for the closest date
    closest_value = df.loc[closest_date, column]
    
    # Calculate the seasonal average
    seasonal_avg = seasonal_average(df, column, target_date)
    
    # Filter data for the current year
    current_year_df = df[df.index.year == target_date.year]
    current_year_avg = current_year_df[column].mean()
    
    # Calculate column metrics
    column_avg = df[column].mean()
    column_max = df[column].max()
    column_min = df[column].min()
    column_std = df[column].std()
    column_median = df[column].median()
    """
    return {
        'Closest Value': closest_value,
        'Seasonal Average': seasonal_avg,
        'Current Year Average': current_year_avg,
        'Column Average': column_avg,
        'Column Max': column_max,
        'Column Min': column_min,
        'Column Std Dev': column_std,
        'Column Median': column_median
    }
    """
    return closest_value, seasonal_avg, current_year_avg, column_avg, column_max, column_min, column_std, column_median


In [ ]:
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

# Initialize trades list
trades2 = []

# Output widget
output11 = widgets.Output()
output33 = widgets.Output()

# Widgets for inputs
receive_currency_dd = widgets.Dropdown(options=[i.upper() for i in spot_library_1.keys()], description='Receive Currency')
receive_structure_text = widgets.Text(description='Receive Structure')
pay_currency_dd = widgets.Dropdown(options=[i.upper() for i in spot_library_1.keys()], description='Pay Currency')
pay_structure_text = widgets.Text(description='Pay Structure')
add_pull_btn_1 = widgets.Button(description='Roll Calc.')
gen_from_1 = dt(dt.today().year - 1, dt.today().month, dt.today().day)
gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
gen_to_1 = dt.today()
df_stats = pd.DataFrame()

def create_tabs(carry_df):
    tab_contents = []
    for month in carry_df.index:
        tab_contents.append(month)
    
    tab = widgets.Tab()
    tab.children = tab_contents
    for i in range(len(tab_contents)):
        tab.set_title(i, carry_df.index[i])
    
    display(tab)
    
# Define the function to handle button click
def add_and_pull_dataf(b):
    global df_stats
    with output11:
        output11.clear_output()
        if receive_structure_text.value != '' and pay_structure_text.value != '':
            trades2.append((receive_currency_dd.value, receive_structure_text.value))
            trades2.append((pay_currency_dd.value, pay_structure_text.value))
            receive_currency_dd.value = list(spot_library_1.keys())[0].upper()
            receive_structure_text.value = ''
            pay_currency_dd.value = list(spot_library_1.keys())[0].upper()
            pay_structure_text.value = ''
        
        master = function_for_bb1(trades2, gen_from_1, gen_to_1)

        temp = function_for_bb1(trades2[:3], gen_from_1, gen_to_1)
        
        # Display the added trades
        max_len = len(trades2)
        y = [f'{i[0]} {i[1]}' for i in trades2] + [float('nan')] * (max_len - len(trades2))
        """
        styled_table = pd.DataFrame(y).style.format('{}', na_rep='').set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
        display(HTML(centered_html))
        """
    with output33:
        output33.clear_output()
        master = function_for_bb1(trades2, gen_from_1, gen_to_1)
        dis = pd.DataFrame({}, index=['CURRENT'])
        dis.loc['CURRENT', master.columns] = master.loc[master.index[-1]]

        # Calculate and display adjusted structures
        adjusted_values = {}
        for trade in trades2:
            adj_struct = adjusted_structure1(trade[1])
            if adj_struct:
                adj_struct = adj_struct[0]
                adj_ticker = swap_structure1(trade[0].lower(), adj_struct, gen_from_1, gen_to_1)
                if adj_ticker is not None and not adj_ticker.empty:
                    dis.loc['CURRENT', f'{trade[0].upper()} {adj_struct}'] = adj_ticker.iloc[-1, 0]
                    adjusted_values[f'{trade[0].upper()} {adj_struct}'] = adj_ticker.iloc[-1, 0]

        # Prepare the final DataFrame for display
        final_columns = [col for pair in zip(master.columns, adjusted_values.keys()) for col in pair]
        final_values = [value for pair in zip(master.iloc[-1], adjusted_values.values()) for value in pair]
        
        rec1, rec2, pay1, pay2 = final_columns[0], final_columns[2], final_columns[1], final_columns[3]
        trades3 = []
        closest_values, seasonal_avgs, current_year_avgs, column_avgs, column_maxs, column_mins, column_stds, column_medians = [], [], [], [],[], [], [], []
        
        for i in final_columns:
            j = i.split()
            trades3.append((j[0], j[1]))

        new_master = function_for_bb1(trades3, gen_from_2, gen_to_1)
        new_master.index = pd.to_datetime(new_master.index)
        new_master["Initial"] = new_master[rec1]-new_master[pay1]-(new_master[rec2]-new_master[pay2])

        for month in range(1, 13):
            new_master["futurerec"] = new_master[rec1] - month * (new_master[rec1] - new_master[rec2]) / 12
            new_master["futurepay"] = new_master[pay1] - month * (new_master[pay1] - new_master[pay2]) / 12

            new_master["future"] = new_master["futurerec"] - new_master["futurepay"] - (new_master[rec2] - new_master[pay2])

            new_master[str(month)] = (new_master["future"] - new_master["Initial"]) * -1

            closest_value, seasonal_avg, current_year_avg, column_avg, column_max, column_min, column_std, column_median = calculate_metrics(new_master, str(month), gen_to_1)
            
            closest_values.append(closest_value)
            seasonal_avgs.append(seasonal_avg)
            current_year_avgs.append(current_year_avg)
            column_avgs.append(column_avg)
            column_maxs.append(column_max)
            column_mins.append(column_min)
            column_stds.append(column_std)
            column_medians.append(column_median)
        
        final_df = pd.DataFrame([final_values], columns=final_columns, index=['CURRENT'])
        data12 = {
            "EUR 5y5y": [final_df.iat[0, 0]],
            "USD 5y5y": [final_df.iat[0, 1]],
            "EUR 4y5y": [final_df.iat[0, 2]],
            "USD 4y5y": [final_df.iat[0, 3]]
        }

        df1 = pd.DataFrame(data12, index=["CURRENT"])
        carry_df = calculate_roll(df1)
        carry_df["Seasonal Avg"] = seasonal_avgs
        carry_df["2024 Avg"] = current_year_avgs
        carry_df["Yearly Avg"] = column_avgs
        carry_df["Max Reached"] = column_maxs
        carry_df["Min Reached"] = column_mins
        carry_df["Std"] = column_stds
        carry_df["Median"] = column_medians

        # Rename the index
        carry_df.index = [f'{i} month' for i in range(1, 13)]

        df_stats = new_master.copy()
        #print(df_stats)
        # Add the title
        title_html = f'<div class="centered-output"><h3>Rec: {rec1}, Pay: {rec2}</h3></div>'
        
        styled_table_roll = carry_df.style.format({
            'Roll': '{:.3f}',
            'Vol Adjusted': '{:.3f}',
            'Seasonal Avg': '{:.3f}',
            '2024 Avg': '{:.3f}',
            'Yearly Avg': '{:.3f}',
            'Max Reached': '{:.3f}',
            'Min Reached': '{:.3f}',
            'Std': '{:.3f}',
            'Median': '{:.3f}',
        }).set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()

        centered_html = f'{title_html}<div class="centered-output"><div class="centered-table">{styled_table_roll}</div></div>'
        display(HTML(centered_html))

        #create_tabs(carry_df)

# Button click event
add_pull_btn_1.on_click(add_and_pull_dataf)

# Display widgets with centered layout
widget_box = widgets.VBox([
    receive_currency_dd, receive_structure_text, pay_currency_dd, pay_structure_text, add_pull_btn_1, output11, output33
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widget_box)

output_stats = widgets.Output()

produce_stats = widgets.Button(description='Expand')
Update_stats = widgets.Button(description='Update Dropdown')           

which_stats = widgets.Dropdown(options=df_stats.columns, description='Series')

input_box_stats = widgets.HBox([which_stats], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
buttons_box_stats = widgets.HBox([Update_stats,produce_stats], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))
output_box_stats = widgets.VBox([output_stats], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(widgets.VBox([input_box_stats, buttons_box_stats, output_box_stats], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')))



def on_button_clicked_stats_1(b):
    with output_stats:
        clear_output()

        # Retrieve the selected series
        series = df_stats[which_stats.value]

        # Extract data for the last week, last month, and last year
        last_week = series[-7:]
        last_month = series[-25:]
        last_year = series[-250:]

        # Generate line plots
        fig, axs = plt.subplots(3, 1, figsize=(10, 15))

        axs[0].plot(last_week.index, last_week.values, label='Last Week', color='blue')
        axs[0].set_title(f'Weekly Move for {which_stats.value}')
        axs[0].set_xlabel('Date')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        axs[1].plot(last_month.index, last_month.values, label='Last Month', color='green')
        axs[1].set_title(f'Monthly Move for {which_stats.value}')
        axs[1].set_xlabel('Date')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        axs[2].plot(last_year.index, last_year.values, label='Last Year', color='red')
        axs[2].set_title(f'Yearly Move for {which_stats.value}')
        axs[2].set_xlabel('Date')
        axs[2].set_ylabel('Value')
        axs[2].legend()

        plt.tight_layout()
        plt.show()

        # Display additional statistics
        display(HTML('<h4>Additional Statistics</h4>'))
        stats_df = pd.DataFrame({
            'Mean': [series.mean(), last_week.mean(), last_month.mean(), last_year.mean()],
            'Standard Deviation': [series.std(), last_week.std(), last_month.std(), last_year.std()],
            'Min': [series.min(), last_week.min(), last_month.min(), last_year.min()],
            'Max': [series.max(), last_week.max(), last_month.max(), last_year.max()],
            'Median': [series.median(), last_week.median(), last_month.median(), last_year.median()]
        }, index=['Original', 'Last Week', 'Last Month', 'Last Year'])
        
        styled_stats_table = stats_df.style.format('{:.3f}').set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        
        centered_stats_html = f'<div class="centered-output"><div class="centered-table">{styled_stats_table}</div></div>'
        display(HTML(centered_stats_html))
    
def on_button_clicked_stats_2(b):
    which_stats.options = df_stats.columns


produce_stats.on_click(on_button_clicked_stats_1)
Update_stats.on_click(on_button_clicked_stats_2)

